<a href="https://colab.research.google.com/github/GoTudering/Deep_Learning_Study/blob/main/Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
print(wine.head())

   alcohol  sugar    pH  class
0      9.4    1.9  3.51    0.0
1      9.8    2.6  3.20    0.0
2      9.8    2.3  3.26    0.0
3      9.8    1.9  3.16    0.0
4      9.4    1.9  3.51    0.0


In [3]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine[['class']].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2,random_state=42)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target,test_size=0.2,random_state=42)

In [6]:
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.8596153846153847


In [9]:
#교차 검증 cross validation
#데이터가 많을 수록 좋은데 검증 세트로 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 듯
#교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복하여 그 점수의 평균을 구함
#k-폴드 교차 검증(k-fold cross validation)은 k 부분으로 나누어 k번 모델 평가하는 것임
#cross_validate()를 사용하면 됨
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.0080235 , 0.006212  , 0.00626421, 0.00622106, 0.00593662]), 'score_time': array([0.00083876, 0.00065613, 0.00064969, 0.00063109, 0.00060201]), 'test_score': array([0.87115385, 0.85192308, 0.87295476, 0.85274302, 0.83830606])}


In [10]:
#이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환하는데 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미
#cross_validate()함수는 기본적으로 5-폴드 교차 검증을 수행
#cv 매개변수에서 폴드 수를 바꿀 수 있음
import numpy as np
print(np.mean(scores['test_score']))

0.8574161545865107


In [14]:
#cross_validate()함수는 훈련 세트를 섞지 않음 그렇기에 분할기splitter를 지정해야함
#cross_validate()함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 가용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용함
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input,train_target,cv=StratifiedKFold())#기본적으로 StratifiedKFold가 지정되어있음
print(np.mean(scores['test_score']))

0.85607018582957


In [15]:
#10-폴드 교차 검증을 하려면
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8583848377056471


In [16]:
#하이퍼파라미터 튜닝

#그리그 서치 Grid Search를 이용하면 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행하여 하이퍼파라이터 튜닝을 할 수 있음
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [17]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [18]:
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [19]:
#검증 점수가 가장 높은 파라미터로 훈련한 모델은 gs객체의 best_estimator_ 속성에 저장되어있음
dt= gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [21]:
#그리드 서치로 찾은 매개변수는 best_params_속성에 저장되어 있음
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [22]:
#각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_result_속성의 'mean_test_score'키에 저장되어있음
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [23]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [26]:
#조금 더 복잡한 매개변수 조합을 탐색해보자
#결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정함
#여기에 max_depth로 트리의 깊이를 제한하고 min_sample_split으로 노드를 나누기 위한 최소 샘플 수를 볼라봄
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)}

In [27]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [28]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
print(np.mean(gs.cv_results_['mean_test_score']))

0.862301492093263


In [31]:
#결국 이렇게 구간 나눠서 그리드 서치하는 것도 내가 그 구간 크기를 정하는거임

#랜덤 서치 Random Search
from scipy.stats import uniform, randint #두 클래스 모두 주어진 범위에서 고르게 값을 뽑음 이를 균등 분포에서 샘플링한다 라고 말함
#randint는 정숫값을 뽑고 uniform은 실숫값을 뽑음
rgen = randint(0,10)
rgen.rvs(10)

array([2, 2, 2, 8, 8, 4, 9, 1, 2, 8])

In [32]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 99, 110, 105, 102, 101,  93, 101,  97,  94,  98]))

In [33]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.98742863, 0.64527914, 0.26475914, 0.64594257, 0.54637927,
       0.10166111, 0.65547512, 0.56465882, 0.9139981 , 0.24022808])

In [34]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [35]:
#샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 저장함
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,
                      n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [36]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [38]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [39]:
dt = gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
